In [29]:
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_lg

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

import pyLDAvis.gensim
from collections import defaultdict



In [3]:
# read water and diarrhea articles
df=pd.read_csv('articles_diarrhea.csv')

In [6]:
news_df=df.loc[~df['text'].isna()]
news_df.reset_index(inplace=True,drop=True)

In [7]:
# removing everything except alphabets`
news_df['clean_doc'] = news_df['text'].str.replace("[^a-zA-Z#]", " ")

# removing short words
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

<ipython-input-7-8aee4566e283>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['clean_doc'] = news_df['text'].str.replace("[^a-zA-Z#]", " ")
<ipython-input-7-8aee4566e283>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))


In [41]:
nlp= spacy.load("en")

# My list of stop words.
stop_list = ["Mrs.","Ms.","say","'s","Mr.",'Prof.','\n']

# Updates spaCy's default stop words list with my additional words. 
nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [42]:
def lemmatizer(doc):
   # take only certain position of sentences
    doc2=[]
    for token in doc:
        if ((token.pos_ != 'PRON') and (token.pos_!= 'SCONJ') and (token.is_currency!=True)  and (token.like_num!=True) and (token.ent_type_ in ['','LAW','PRODUCT','EVENT'] ) and not token.is_digit and (token.pos_ != 'CCONJ') and (token.pos_ != 'DET') and (token.pos_ != 'ADP') and (token.pos_ != 'PART') and (token.pos_ != 'ADV') and (token.pos_ != 'AUX') ):
            doc2.append(token.lemma_.lower())
    doc2 = u' '.join(doc2)
    return nlp.make_doc(doc2)
    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# The add_pipe function appends our functions to the default pipeline.
nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

In [43]:
doc_list = []
# Iterates through each article in the corpus.
for doc in tqdm(newsdoc):
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(doc)
    doc_list.append(pr)   

<ipython-input-43-77cc3c2c96cf>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for doc in tqdm(newsdoc):


In [45]:
frequency = defaultdict(int)
for text in doc_list:
    for token in text:
        frequency[token] += 1

doc_list = [
    [token for token in text if (frequency[token] > 1) and not('\n' in token) and not ('Prof.' in token)]
    for text in doc_list
]

pprint(doc_list[0])

['listen',
 'article',
 'professor',
 'director',
 'health',
 'development',
 'geographer',
 'research',
 'interest',
 'underscore',
 'sanitation',
 'issue',
 'cost',
 'population',
 'defecate',
 'equate',
 '5.4million',
 'population',
 'estimate',
 'accord',
 'prof',
 'mariwah',
 'open',
 'defecation',
 'situation',
 'people',
 'choose',
 'ease',
 'beach',
 'gutter',
 'house',
 'consistent',
 'habit',
 'ask',
 'description',
 'decide',
 'ease',
 'nearby',
 's',
 'develop',
 'stomach',
 'upset',
 'travel',
 'open',
 'defecation',
 'count',
 'calculate',
 'people',
 'practice',
 'open',
 'defecation',
 'consistent',
 'habit',
 'person',
 'happen',
 'individual',
 'alternative',
 'moment',
 'explain',
 'cause',
 'open',
 'defecation',
 'cultural',
 'social',
 'spiritual',
 'economic',
 'dimension',
 'system',
 'toilet',
 'build',
 'outskirt',
 'town',
 'people',
 'build',
 'house',
 'incorporate',
 'toilet',
 'facility',
 'add',
 'jurisdiction',
 'people',
 'defecate',
 'provide',
 'toil

In [46]:
# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

In [47]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=5, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [48]:
pprint(lda_model.print_topics(num_words=20))

[(0,
  '0.020*"people" + 0.014*"water" + 0.014*"cholera" + 0.012*"child" + '
  '0.009*"health" + 0.009*"case" + 0.008*"camp" + 0.007*"disease" + '
  '0.007*"need" + 0.007*"area" + 0.006*"outbreak" + 0.006*"food" + '
  '0.006*"country" + 0.006*"refugee" + 0.006*"aid" + 0.005*"humanitarian" + '
  '0.005*"include" + 0.005*"diarrhoea" + 0.005*"government" + 0.005*"provide"'),
 (1,
  '0.054*"death" + 0.041*"disease" + 0.025*"age" + 0.022*"cancer" + '
  '0.019*"cause" + 0.014*"professor" + 0.014*"rate" + 0.013*"man" + '
  '0.012*"die" + 0.011*"suicide" + 0.011*"people" + 0.010*"killer" + '
  '0.010*"circulatory" + 0.010*"life" + 0.010*"infectious" + 0.009*"group" + '
  '0.008*"young" + 0.008*"new" + 0.008*"injury" + 0.008*"account"'),
 (2,
  '0.018*"good" + 0.012*"beach" + 0.009*"use" + 0.006*"poor" + 0.005*"include" '
  '+ 0.005*"site" + 0.005*"health" + 0.005*"ship" + 0.005*"time" + '
  '0.005*"water" + 0.004*"study" + 0.004*"swimming" + 0.004*"food" + '
  '0.004*"datum" + 0.003*"report" +

In [49]:
pyLDAvis.enable_notebook()

In [50]:
pyLDAvis.gensim.prepare(lda_model, corpus, words)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.087689 -0.051575       1        1  33.218468
0     -0.090650 -0.143732       2        1  29.262993
3     -0.096673 -0.012740       3        1  21.734472
2     -0.087967  0.209116       4        1   9.780899
1      0.362979 -0.001069       5        1   6.003175, topic_info=         Term          Freq         Total Category  logprob  loglift
598     death  16860.000000  16860.000000  Default  30.0000  30.0000
185   disease  25300.000000  25300.000000  Default  29.0000  29.0000
158     water  68230.000000  68230.000000  Default  28.0000  28.0000
1149      age   6184.000000   6184.000000  Default  27.0000  27.0000
22      cause  13396.000000  13396.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
114    people   2094.531250  36535.515625   Topic5  -4.5399  -0.0461
855      lead   1375.116333   7036.951660   Topic5  -4.9607   1.1802
224      road   1131.187378   3402.677246   Topic5  -5.1560   1.7116
451     woman   1269.233154   8231.599609   Topic5  -5.0408   0.9433
23      child   1146.349609  30110.968750   Topic5  -5.1427  -0.4554

[446 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
6092      5  0.995362       0-
6093      5  0.997212      15-
6094      5  0.998379  3½-fold
6096      5  0.995171     88db
6650      1  0.010148        =
...     ...       ...      ...
324       2  0.151572    young
324       3  0.137768    young
324       4  0.010491    young
324       5  0.453612    young
5417      4  0.997217        ®

[942 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 4, 3, 2])